# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

# Import Data & Libraries

In [1]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [2]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_vals(rows, col_index):
    """Mengembalikan set nilai unik pada kolom ke-`col_index`."""
    return set([row[col_index] for row in rows])
# contoh penggunaan
print(unique_vals(training_data, 0))  # kolom warna
print(unique_vals(training_data, 1))  # kolom diameter
print(unique_vals(training_data, 2))  # kolom label

{'Yellow', 'Red', 'Green'}
{1, 3}
{'Grape', 'Lemon', 'Apple'}


In [3]:
# fungsi Menghitung jumlah unique value dari suatu kolom (kolom label)
def class_counts(rows):
    """
    Mengembalikan dict {label: jumlah} untuk kolom terakhir pada data.
    """
    counts = {}  # dictionary kosong
    for row in rows:
        label = row[-1]        # ambil kolom terakhir (label)
        counts[label] = counts.get(label, 0) + 1
    return counts

# contoh penggunaan
print(class_counts(training_data))


{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [4]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    """Mengembalikan True jika value bertipe int atau float, False jika tidak."""
    return isinstance(value, (int, float))

# contoh penggunaan
print(is_numeric(3))        # True
print(is_numeric(3.5))      # True
print(is_numeric("Green"))  # False
print(is_numeric("7"))      # False (string, bukan int/float)


True
True
False
False


In [5]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column   # indeks kolom, misal 0 = color
        self.value = value     # nilai yang dibandingkan

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        # example = satu baris data, misal ['Green', 3, 'Apple']
        val = example[self.column]
        # jika nilai pertanyaan numerik, gunakan perbandingan >=
        if is_numeric(val):
            return val >= self.value
        else:
            # untuk data kategorik, cek kesamaan
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = ">=" if is_numeric(self.value) else "=="
        return f"Apakah {header[self.column]} {condition} {self.value}?"


# --------- contoh penggunaan 1 ----------
q1 = Question(0, "Green")  # tanya apakah warna == Green
print(q1)                  # => Apakah color == Green?
print(q1.match(['Green', 3, 'Apple']))  # True
print(q1.match(['Red', 1, 'Grape']))    # False

q2 = Question(1, 3)        # tanya apakah diameter >= 3
print(q2)                  # => Apakah diameter >= 3?
print(q2.match(['Yellow', 4, 'Apple'])) # True
print(q2.match(['Red', 1, 'Grape']))    # False


Apakah color == Green?
True
False
Apakah diameter >= 3?
True
False


In [6]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    """Mengembalikan dua list: (true_rows, false_rows)"""
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

# -------- contoh penggunaan ----------
q = Question(0, "Green")  # pertanyaan: apakah color == Green?
true_rows, false_rows = partition(training_data, q)

print("True rows:")
for r in true_rows:
    print(r)

print("\nFalse rows:")
for r in false_rows:
    print(r)


True rows:
['Green', 3, 'Apple']

False rows:
['Yellow', 3, 'Apple']
['Red', 1, 'Grape']
['Red', 1, 'Grape']
['Yellow', 3, 'Lemon']


**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [7]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    """
    Gini impurity = 1 - Σ (p_i)^2
    di mana p_i adalah proporsi masing-masing class.
    """
    counts = class_counts(rows)
    impurity = 1.0
    total = float(len(rows))
    for lbl, cnt in counts.items():
        prob_of_lbl = cnt / total
        impurity -= prob_of_lbl ** 2
    return impurity

# -------- contoh penggunaan ----------
print("Gini training_data:", gini(training_data))

# Contoh subset
subset = [
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape']
]
print("Gini subset Grape-only:", gini(subset))


Gini training_data: 0.6399999999999999
Gini subset Grape-only: 0.0


**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [8]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    """
    left, right : hasil partition (list of rows)
    current_uncertainty : gini dari dataset sebelum split
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# -------- contoh penggunaan ----------
# misal kita pisahkan training_data dengan pertanyaan tertentu
q = Question(0, "Green")                 # Apakah color == Green?
true_rows, false_rows = partition(training_data, q)

current_uncertainty = gini(training_data)
gain = info_gain(true_rows, false_rows, current_uncertainty)

print("Information Gain untuk pertanyaan", q, "=", gain)


Information Gain untuk pertanyaan Apakah color == Green? = 0.1399999999999999


In [9]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    """
    Mengembalikan tuple (best_gain, best_question)
    """
    best_gain = 0        # nilai information gain terbaik
    best_question = None # pertanyaan terbaik
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # semua kolom kecuali label

    for col in range(n_features):
        # ambil nilai unik di kolom ini
        values = set([row[col] for row in rows])

        for val in values:
            question = Question(col, val)

            # splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # hitung information gain
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # simpan jika lebih baik
            if gain > best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


# -------- contoh penggunaan ----------
gain, question = find_best_split(training_data)
print("Best Gain:", gain)
print("Best Question:", question)


Best Gain: 0.37333333333333324
Best Question: Apakah color == Red?


# Fungsi Decision Tree

In [10]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        # dictionary {label: jumlah} untuk semua contoh di subset ini
        self.predictions = class_counts(rows)

# -------- contoh penggunaan ----------
leaf_node = Leaf([
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape']
])

print(leaf_node.predictions)


{'Apple': 2, 'Grape': 1}


In [11]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self, question, true_branch, false_branch):
        """
        question     : objek Question yang dipakai di node ini
        true_branch  : subtree (Decision_Node/Leaf) untuk data yang match
        false_branch : subtree (Decision_Node/Leaf) untuk data yang tidak match
        """
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


# -------- contoh penggunaan ----------
# misal kita buat leaf secara manual
leaf_true = Leaf([['Green', 3, 'Apple']])
leaf_false = Leaf([['Red', 1, 'Grape']])

# buat decision node dengan pertanyaan “color == Green?”
example_question = Question(0, "Green")
root_node = Decision_Node(example_question, leaf_true, leaf_false)

print("Pertanyaan di root:", root_node.question)
print("Prediksi jika True branch:", root_node.true_branch.predictions)
print("Prediksi jika False branch:", root_node.false_branch.predictions)


Pertanyaan di root: Apakah color == Green?
Prediksi jika True branch: {'Apple': 1}
Prediksi jika False branch: {'Grape': 1}


In [12]:
# membangun decision tree secara rekursif
def build_tree(rows):
    """
    Mengembalikan root dari decision tree.
    """
    # 1. Cari pertanyaan terbaik untuk split data saat ini
    gain, question = find_best_split(rows)

    # 2. Basis rekursi: jika tidak ada information gain,
    #    buat Leaf (node daun) dengan distribusi label
    if gain == 0:
        return Leaf(rows)

    # 3. Jika ada pertanyaan terbaik, pisahkan data
    true_rows, false_rows = partition(rows, question)

    # 4. Rekursif ke cabang true dan false
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)

    # 5. Kembalikan node keputusan
    return Decision_Node(question, true_branch, false_branch)


# -------- contoh penggunaan ----------
tree = build_tree(training_data)
print("Tree berhasil dibangun!")
print("Root Question:", tree.question)
print("True branch type:", type(tree.true_branch).__name__)
print("False branch type:", type(tree.false_branch).__name__)


Tree berhasil dibangun!
Root Question: Apakah color == Red?
True branch type: Leaf
False branch type: Decision_Node


In [13]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):
    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        print(spacing + "Prediksi:", node.predictions)
        return

    # mencetak pertanyaan pada node saat ini
    print(spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


# -------- contoh penggunaan ----------
tree = build_tree(training_data)
print_tree(tree)


Apakah color == Red?
--> True:
  Prediksi: {'Grape': 2}
--> False:
  Apakah color == Yellow?
  --> True:
    Prediksi: {'Apple': 1, 'Lemon': 1}
  --> False:
    Prediksi: {'Apple': 1}


In [14]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):
    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)


# -------- contoh penggunaan ----------
tree = build_tree(training_data)

# klasifikasi beberapa contoh
for test_row in testing_data:
    prediction = classify(test_row, tree)
    print(f"Data: {test_row} => Prediksi: {prediction}")


Data: ['Green', 3, 'Apple'] => Prediksi: {'Apple': 1}
Data: ['Yellow', 4, 'Apple'] => Prediksi: {'Apple': 1, 'Lemon': 1}
Data: ['Red', 2, 'Grape'] => Prediksi: {'Grape': 2}
Data: ['Red', 1, 'Grape'] => Prediksi: {'Grape': 2}
Data: ['Yellow', 3, 'Lemon'] => Prediksi: {'Apple': 1, 'Lemon': 1}


In [15]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    """
    counts : dict {label: jumlah}
    return : dict {label: 'persentase%'}
    """
    total = sum(counts.values())
    probs = {}
    for lbl, cnt in counts.items():
        percentage = int(cnt / total * 100)
        probs[lbl] = f"{percentage}%"
    return probs


# -------- contoh penggunaan ----------
example_counts = {'Apple': 2, 'Grape': 1, 'Lemon': 1}
print(print_leaf(example_counts))


{'Apple': '50%', 'Grape': '25%', 'Lemon': '25%'}


# Predict Using Decision Tree

In [17]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
for row in testing_data:
    prediction = classify(row, tree)             # distribusi label
    predicted_label = max(prediction, key=prediction.get)  # ambil label mayoritas
    actual_label = row[-1]                       # label asli (kolom terakhir)
    print(f"Data: {row[:-1]} | Prediksi: {predicted_label} | Asli: {actual_label}")



Data: ['Green', 3] | Prediksi: Apple | Asli: Apple
Data: ['Yellow', 4] | Prediksi: Apple | Asli: Apple
Data: ['Red', 2] | Prediksi: Grape | Asli: Grape
Data: ['Red', 1] | Prediksi: Grape | Asli: Grape
Data: ['Yellow', 3] | Prediksi: Apple | Asli: Lemon
